In [ ]:
! pip install langchain-mcp-adapters langgraph "langchain[anthropic]" langgraph-swarm

In [1]:
# Imports
from langchain_anthropic import ChatAnthropic
from langchain_mcp_adapters.client import MultiServerMCPClient

from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

planner_prompt = """
<Task>
You will help plan the steps to implement a LangGraph application. 
</Task>

<Instructions>
1. Reflect on the user's request. 
2. Use the list_doc_sources tool to fetch and the fetch_docs tool to read the llms.txt file.
3. Identify documents that are relevant to the user's request.
4. Ask follow-up questions to help refine the project scope and narrow the set of documents to be used for the project.
5. When the project scope is clear produce a short description of the project with relevant URLs.
6. Finally, transfer to transfer_to_researcher_agent.
</Instructions>
"""

researcher_prompt = """
<Task>
You will perform research on the project scope. 
</Task>

<Instructions>
1. Reflect on the project scope and provided URLs from the planner.
2. Use the fetch_docs tool to fetch and read each URL.
3. Use the information in these URLs to implement the solution to the user's request.
4. If you need further clarification or additional sources to implement the solution, transfer to transfer_to_planner_agent.
</Instructions>
"""

# LLM
model = ChatAnthropic(model="claude-3-7-sonnet-latest")

# Handoff tools
transfer_to_planner_agent = create_handoff_tool(
    agent_name="planner_agent",
    description="Transfer user to the planner_agent to address clarifying questions or help them plan the steps to complete the user's request."
)
transfer_to_researcher_agent = create_handoff_tool(
    agent_name="researcher_agent",
    description="Transfer user to researcher_agent to perform research on the user's request."
)

# TODO: Move to configuration
#configurable = Configuration.from_runnable_config(config)
#llms_txt_urls = configurable.llms_txt
llms_txt_urls = "LangGraph:https://langchain-ai.github.io/langgraph/llms.txt"


ModuleNotFoundError: No module named 'langchain_anthropic'

In [7]:
# Input 
messages = {"role": "user", "content": "Create a prompt chain that makes and improves a joke based on the user's input."}

In [ ]:
# Research and planning tools 
async with MultiServerMCPClient(
    {
        "research-server": {
            "command": "npx",
            "args": ["@playwright/mcp"],
            "transport": "stdio",
            "env": {
                "PATH": "/Users/rlm/.cursor/extensions/ms-python.python-2024.12.3-darwin-arm64/python_files/deactivate/zsh:/Users/rlm/Desktop/Code/langgraph-swarm/.venv/bin:/Users/rlm/.bun/bin:/Users/rlm/.poetry/bin:/Users/rlm/Library/Python/3.13/bin:/opt/homebrew/bin:/opt/homebrew/sbin:/usr/local/bin:/System/Cryptexes/App/usr/bin:/usr/bin:/bin:/usr/sbin:/sbin:/Library/TeX/texbin:/Users/rlm/.cargo/bin:/Users/rlm/miniforge3/condabin:/Users/rlm/.local/bin"
            }
        },
        "planning-server": {
            "command": "uvx",
            "args": [
            "--from",
            "mcpdoc",
            "mcpdoc",
            "--urls",
            llms_txt_urls,
            "--transport",
            "stdio",
            "--port",
            "8081",
            "--host",
            "localhost"
            ],
            "transport": "stdio",
        }
    }

) as client:
    # Planner agent
    planner_agent = create_react_agent(model,
                                       prompt=planner_prompt, 
                                       tools=client.server_name_to_tools["planning-server"].append(transfer_to_researcher_agent),
                                       name="planner_agent") 

    # Researcher agent
    researcher_agent = create_react_agent(model, 
                                          prompt=researcher_prompt, 
                                          tools=client.server_name_to_tools["research-server"].append(transfer_to_planner_agent),
                                          name="researcher_agent") 

    # Swarm
    agent_swarm = create_swarm([planner_agent, researcher_agent], default_active_agent="planner_agent")

    # app = agent_swarm.compile(config_schema=Configuration)
    agent = agent_swarm.compile()
    response = await agent.ainvoke({"messages": messages})


In [ ]:
for m in agent_response['messages']:
    m.pretty_print()